Learn to predict the number of real solutions of the tritangent circles to three conics.

In [ ]:
#TODO inspect predictions
#TODO check distribution of real counts in training data
#TODO ???

In [ ]:
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import pandas as pd
import numpy as np

In [ ]:
# Import Data
path = '/Users/las/Documents/GitHub/REU/hill_climb_w_solution_counts.xlsx'
df = pd.read_excel(path)

batch_1 = df[[col for col in df.columns if col[0].isupper()]].rename(str.lower, axis='columns')
batch_2 = df[[col for col in df.columns if col[0].islower()]].rename(str.lower, axis='columns')
batch_2.insert(0, "counts", np.array(batch_1['counts']) - 2)
batch_2[batch_2['counts'] < 0] = 0
data = batch_1.append(batch_2).reset_index().drop('index', axis=1)

In [ ]:
data.sample()

In [ ]:
# Create Training and Validation Data
count = np.array(data['counts'].values.tolist(), dtype=int)
param = np.array(data.drop('counts', axis=1).values.tolist(), dtype=np.float64)
n = int(0.95*len(count))

count_train, count_val = count[:n], count[n:]
param_train, param_val = param[:n], param[n:]

In [ ]:
# Set Up Network Architecture
model = keras.Sequential([
    keras.layers.Dense(units=32, activation='relu', input_shape=(18,)),
    keras.layers.Dense(units=64, activation='relu'),    
    keras.layers.Dense(units=96, activation='relu'),
    keras.layers.Dense(units=1, activation='softmax')
])

model.summary()

In [ ]:
# cce = keras.losses.CategoricalCrossentropy()
sce = keras.losses.SparseCategoricalCrossentropy()

model.compile(optimizer='RMSprop', 
              loss=sce,
              metrics=[sce, 'accuracy'])

In [ ]:
# Train model
history = model.fit(param_train,
                    count_train,
                    verbose=2,
                    epochs=25,
                    batch_size=64,
                    validation_data=(
                        param_val,
                        count_val
                    )
)

In [ ]:
# Graphs
fig = plt.figure()
plt.title('Model Loss')
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss'])
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.plot()

In [ ]:
# Inspect Predictions
fig = plt.figure()
prediction = model.predict(param_val)
plt.plot(prediction, 'o')
plt.show()

In [ ]:
print(prediction[:5])